In [1]:
from pypdf import PdfReader


# Load the PDF file
pdf = PdfReader("data/icd/icd10gm2026syst_referenz_20250912.pdf")

meta = pdf.metadata

In [2]:
page = pdf.pages[600].extract_text()
page

'  S13.-  Luxation, Verstauchung und Zerrung von Gelenken und Bändern in \nHalshöhe\nBenutze eine zusätzliche Schlüsselnummer aus S11.84!-S11.89! zusammen mit S13.0-S13.3, um \nden Schweregrad des Weichteilschadens bei einer Luxation zu verschlüsseln.\nEine Verletzung des zervikalen Rückenmarks ist zusätzlich mit S14.- zu verschlüsseln.\nDas Vorliegen von Halswirbelfrakturen bei einer Luxation ist zusätzlich mit S12.- zu verschlüsseln. \nIst die Zahl der zusammen mit der Luxation gebrochenen Halswirbel nicht bekannt, so ist die \nFraktur auf der höchsten Ebene zu verschlüsseln.\nExkl.: Ruptur oder Verlagerung (nichttraumatisch) einer zervikalen Bandscheibe (M50.-)\nVerstauchung und Zerrung von Muskeln und Sehnen in Halshöhe (S16)\nS13.0 Traumatische Ruptur einer zervikalen Bandscheibe\nS13.1- Luxation eines Halswirbels\nS13.10 Höhe nicht näher bezeichnet\nS13.11 C1/C2\nS13.12 C2/C3\nS13.13 C3/C4\nS13.14 C4/C5\nS13.15 C5/C6\nS13.16 C6/C7\nS13.17 C7/T1\nS13.18 Sonstige\nS13.2 Luxation so

In [3]:
import re

chunks = []

for page in pdf.pages:
    pattern = r'([A-Z][0-9][0-9](?:\.[0-9A-Z]+)?)'

    extract = page.extract_text()
    codes = re.findall(pattern, extract)

    for i, code in enumerate(codes):
        start = extract.find(code)
        end = extract.find(codes[i+1], start) if i+1 < len(codes) else len(extract)
        section = extract[start:end].strip()
        chunks.append({"code": code, "text": section, "page_number": pdf.pages.index(page)+1})

In [4]:
metadata = {
 'Author': meta.get('/Author', ''),
 'CreationDate': meta.get('/CreationDate', ''),
 'Creator': meta.get('/Creator', ''),
 'ModDate': meta.get('/ModDate', ''),
 'Title': meta.get('/Title', ''),
}

In [5]:
for code in chunks:
    metadata.update({'ICD_Code': code['code'], 'Page_Number': code['page_number']})

In [12]:
from llama_index.core.schema import TextNode

# Parent Docs
parent_docs = []

for i, page in enumerate(pdf.pages):
    parent_docs.append(
        TextNode(
            text=page.extract_text(),
            metadata = {
                'Author': meta.get('/Author', ''),
                'CreationDate': meta.get('/CreationDate', ''),
                'Creator': meta.get('/Creator', ''),
                'ModDate': meta.get('/ModDate', ''),
                'Title': meta.get('/Title', ''),
                'Page': pdf.pages.index(page) + 1
            }
        )
    )

# Child Docs
child_docs = []

for chunk in chunks:
    child_docs.append(
        TextNode(
            text=chunk['text'],
            metadata = {
                'ICD_Code': chunk['code'],
                'Page': chunk['page_number'],
                'Title': meta.get('/Title', '')
            }
        )
    )

In [14]:
from llama_index.core.schema import NodeRelationship, RelatedNodeInfo

for parent in parent_docs:
    for child in child_docs:
        if child.metadata['Page'] == parent.metadata['Page']:
            
            child.relationships[NodeRelationship.PARENT] = RelatedNodeInfo(
                node_id=parent.node_id
            )
            parent.relationships.setdefault(NodeRelationship.CHILD, [])
            parent.relationships[NodeRelationship.CHILD].append(
                RelatedNodeInfo(node_id=child.node_id)
            )

In [75]:
# Now you can use parent_docs and child_docs as needed
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize the embedding model
embeddings = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", device="cuda")
Settings.embed_model = embeddings

storage_content = StorageContext.from_defaults()

all_nodes = parent_docs + child_docs

index = VectorStoreIndex.from_documents(
    embeddings=embeddings,
    documents=all_nodes,
    storage_context=storage_content,
    show_progress=True
)

Generating embeddings: 100%|██████████| 1030/1030 [00:01<00:00, 665.18it/s]


In [81]:
retriever = index.as_retriever(similarity_top_k=20)
result = retriever.retrieve("Welche ICD Codes für Diabetes mellitus gibt es?")
result

[NodeWithScore(node=TextNode(id_='c03eaf25-f912-4511-95f6-c16a8d207410', embedding=None, metadata={'ICD_Code': 'E07.9', 'Page': 138, 'Title': 'ICD-10-GM Version 2026 Systematisches Verzeichnis - Referenzfassung'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8b22fc4e-fdfd-43f4-a970-132d3cd3f51c', node_type='1', metadata={'ICD_Code': 'E07.9', 'Page': 138, 'Title': 'ICD-10-GM Version 2026 Systematisches Verzeichnis - Referenzfassung'}, hash='d60f1e19fecfa1f10234dfc71347e0338f5ae98d4bbec64080c74dfb9b725b49')}, metadata_template='{key}: {value}', metadata_separator='\n', text='E07.9 Krankheit der Schilddrüse, nicht näher bezeichnet\nDiabetes mellitus\n(', mimetype='text/plain', start_char_idx=0, end_char_idx=75, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=0.7991898988268737),
 NodeWithScore(node=TextNode(id_='81d3d549-9481-4232-88cf-4e8c36bbce9e', embedding=None, meta

In [84]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOllama(model="qwen3:4b", temperature=0.1)

In [85]:
context = "\n\n".join([doc.text for doc in result])

prompt = PromptTemplate.from_template(
    "Basierend auf dem folgenden Kontext beantworte die Frage präzise:\n\n"
    "Lasse keine Informationen weg und erfinde keine Antworten.\n\n"
    "Kontext:\n{context}\n\n"
    "Frage: {question}\n\n"
    "Antwort:"
)

qa = prompt | model | StrOutputParser()

qa_result = qa.invoke({'context': context, 'question': "Welche ICD Codes für Diabetes mellitus gibt es?"})
qa_result


'Basierend auf der gegebenen Information und den ICD-10-Klassifikationen für Diabetes mellitus, sind die folgenden ICD-Codes spezifisch für Diabetes mellitus selbst (d.h. die Krankheit, nicht Komplikationen oder Prozeduren):\n\n1. **E14.-** – Unspezifiziertes Diabetes mellitus (allgemeiner Code für Diabetes mellitus ohne weitere Spezifikation)  \n2. **O24.9** – Diabetes mellitus in der Schwangerschaft, unspezifiziert  \n3. **O24.3** – Vorhergegangenes Diabetes mellitus, unspezifiziert  \n4. **O24.1** – Vorhergegangenes Diabetes mellitus, Typ 2  \n5. **R73.08** – Diabetes mellitus bei Neugeborenen  \n\n### Begründung:\n- **E14.-** ist der Code für Diabetes mellitus ohne weitere Spezifikation (z.B. Typ 1/2 oder Schwangerschaft).  \n- **O24.9**, **O24.3** und **O24.1** sind spezifische Codes für Diabetes mellitus in der Schwangerschaft oder bei Vorhergegangenem Diabetes (inkl. Typ 2).  \n- **R73.08** bezieht sich auf Diabetes mellitus bei Neugeborenen (eine spezifische Situation).  \n- An

In [86]:
from IPython.display import display, Markdown

display(Markdown(f"**Antwort:** {qa_result}"))

**Antwort:** Basierend auf der gegebenen Information und den ICD-10-Klassifikationen für Diabetes mellitus, sind die folgenden ICD-Codes spezifisch für Diabetes mellitus selbst (d.h. die Krankheit, nicht Komplikationen oder Prozeduren):

1. **E14.-** – Unspezifiziertes Diabetes mellitus (allgemeiner Code für Diabetes mellitus ohne weitere Spezifikation)  
2. **O24.9** – Diabetes mellitus in der Schwangerschaft, unspezifiziert  
3. **O24.3** – Vorhergegangenes Diabetes mellitus, unspezifiziert  
4. **O24.1** – Vorhergegangenes Diabetes mellitus, Typ 2  
5. **R73.08** – Diabetes mellitus bei Neugeborenen  

### Begründung:
- **E14.-** ist der Code für Diabetes mellitus ohne weitere Spezifikation (z.B. Typ 1/2 oder Schwangerschaft).  
- **O24.9**, **O24.3** und **O24.1** sind spezifische Codes für Diabetes mellitus in der Schwangerschaft oder bei Vorhergegangenem Diabetes (inkl. Typ 2).  
- **R73.08** bezieht sich auf Diabetes mellitus bei Neugeborenen (eine spezifische Situation).  
- Andere Codes wie **U69.75** (Diabetes mellitus, Typ 1 in der Schwangerschaft) sind **nicht** für die Krankheit selbst, sondern für Komplikationen der Schwangerschaft (U69 ist die Kategorie für "Komplikationen der Schwangerschaft").  
- **Z83.3** (Diabetes mellitus in der Familienanamnese) und **Z13.18** (Prozeduren zur Diagnose) sind nicht Krankheitscodes, sondern Anamnese- oder Prozedurcodes.  

Diese fünf Codes sind die **eindeutig für Diabetes mellitus selbst** definierten ICD-10-Codes, wie sie im gegebenen Kontext beschrieben werden.  

**Antwort:** E14.-, O24.9, O24.3, O24.1, R73.08